# Long Form Question Answering with ELI5 and Wikipedia  

---  

### Table of Contents  

1. [Introduction](#intro)  
2. [Task and Data Description](#task_description)  
3. [Retrieving Support Documents](#retrieval)  
    a. [Sparse Retrieval with ElasticSearch](#elasticsearch)  
    b. [Training a Dense Retriever with ELI5 and in-batch Negatives](#dense_train)  
    c. [Using a Trained Dense Retriever](#dense_use)  
    d. [Retriever Evaluation](#dense_eval)  
4. [Answer Generation Model](#generation)  
    a. [Conditional Generation with Seq2seq Models](#seq2seq_presentation)  
    b. [Fine-Tuning Seq2seq Models](#seq2seq_train)  
5. [Conclusion](#conclusion)  


---

<img src="images/choco_bis.svg" width="900" align="center"/>  


## Introduction
<a id='intro'></a>

Imagine that you are taken with a sudden desire to understand **how the fruit of a tropical tree gets transformed into chocolate bars**, or want to understand **the role of fever in the human body's immune response**: how would you go about finding that information?

If your specific question has already been asked and provided a clear and succint answer on one of the many question answering platforms answering on the Internet (such as [**Quora**](https://www.quora.com/How-is-chocolate-made), [**Reddit**](https://www.reddit.com/user/ex_5_libris/comments/9c8gb1/chocolate_how_chocolate_is_made/), or [**Yahoo Answers**](https://answers.yahoo.com/question/index?qid=20070615082202AArsYN1)), you're in luck: modern search engine will probably take you to that pre-existing answer pretty reliably. Otherwise, the process will be a little more involved. You will likely have to collect relevant information from a variety of sources, figure out how these pieces of knowledge fit together in relation to your query, and synthetize a narrative that answers your initial question.

Now, wouldn't it be great if your computer could do all of that for you: **gather** the right sources, **synthetize** the information, and **write up** an easy-to-read summary of the relevant points? The bad news is: such a system isn't quite available yet, at least not one that can provide *reliable* information in its summary. The good news on the other hand: a number of recent advances in natural language understanding and generation have made working toward solving this task much easier. These advances include progress in the pre-training (e.g. [BART](https://arxiv.org/abs/1910.13461), [T5](https://arxiv.org/abs/1910.10683)) and evaluation (e.g. for [factuality](https://arxiv.org/abs/2004.04228)) of sequence-to-sequence models used for conditional text generation, new ways to use these models to find information in Wikipedia (e.g. [REALM](https://kentonl.com/pub/gltpc.2020.pdf), [DPR](https://arxiv.org/abs/2004.04906)), and new [training datasets](https://arxiv.org/abs/1907.09190).

**In this notebook,** we show how we can take advantage of some of these recent works to train a **long form question answering** system which takes in a question, fetches 10 relevant passages from a [Wikipedia snapshot](https://www.aclweb.org/anthology/2020.lrec-1.297/), and writes a multi-sentence answer based on the question and retrieved passages. Follow along to learn about the steps involved and read some background on the state of the art for some related tasks, or go straight to the:  
## [**Live Demo!**](http://35.226.96.115:8080/)  
(And don't forget to scroll down on the left sidebar to show all of the generation options!)

#### Preliminaries  

The implementation presented here relies on the [HuggingFace](https://huggingface.co/) [🤗transformers](https://github.com/huggingface/transformers) and [🤗nlp](https://github.com/huggingface/nlp) libraries. Wikipedia indexing relies on [ElasticSearch](https://www.elastic.co/elasticsearch) with its [python bindings](https://github.com/elastic/elasticsearch-py) for the sparse version, and [faiss](https://github.com/facebookresearch/faiss/) for the dense version. You can get all of these by running:
> pip install elasticsearch  
> pip install faiss_gpu  
> pip install nlp  
> pip install transformers  
>  
> wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.7.1-linux-x86_64.tar.gz  
> tar -xzvf elasticsearch-7.7.1-linux-x86_64.tar.gz  

The training relies on two datasets: [ELI5](https://arxiv.org/abs/1907.09190), a processed version of the [r/explainlikeimfive](https://www.reddit.com/r/explainlikeimfive/) subreddit, and the [Wiki40b](https://www.aclweb.org/anthology/2020.lrec-1.297/) Wikipedia image. Downloading these and splitting Wikipedia into snippets for indexing can take a long time (up to 72 hours for the ELI5 dataset creation, which has to filter through all of the Reddit dumps). We suggest that you start by downloading and pre-processing these as follows before doing anything else:

In [1]:
import nlp
from eli5_utils import *

wiki40b_snippets = nlp.load_dataset('wiki_snippets', name='wiki40b_en_100_0', experimental=True)['train']
eli5 = nlp.load_dataset('explainlikeimfive', name='LFQA_reddit', experimental=True)

## Task and Data Description
<a id='task_description'></a>

The task of Long Form Question Answering

In [2]:
eli5['test_eli5'][12345]

{'q_id': '8houtx',
 'title': 'Why does water heated to room temperature feel colder than the air around it?',
 'selftext': '',
 'document': '',
 'subreddit': 'explainlikeimfive',
 'answers': {'a_id': ['dylcnfk', 'dylcj49'],
  'text': ["Water transfers heat more efficiently than air. When something feels cold it's because heat is being transferred from your skin to whatever you're touching. Since water absorbs the heat more readily than air, it feels colder.",
   "Air isn't as good at transferring heat compared to something like water or steel (sit on a room temperature steel bench vs. a room temperature wooden bench, and the steel one will feel more cold).\n\nWhen you feel cold, what you're feeling is heat being transferred out of you.  If there is no breeze, you feel a certain way.  If there's a breeze, you will get colder faster (because the moving air is pulling the heat away from you), and if you get into water, its quite good at pulling heat from you.   Get out of the water and ha

## Retrieving Support Documents
<a id='retrieval'></a>

The first question is...

### Sparse Retrieval with ElasticSearch
<a id='elasticsearch'></a>

The traditional approach until...  

First, let's create a dense index

In [2]:
from eli5_utils import *

es_client = Elasticsearch([{'host': 'localhost', 'port': '9200'}])
if not es_client.indices.exists('wiki40b_snippets_100w'):
    make_es_index_snippets(es_client, wiki40b_snippets, index_name='wiki40b_snippets_100w')

Now let's test for one of the ELI5 questions:

In [6]:
question = eli5['test_eli5'][12345]['title']
doc, res_list = query_es_index(question, es_client, index_name='wiki40b_snippets_100w', n_results=10)

print(question)
print('-----\n')
for res in res_list:
    print("{}: \n  {}\n".format(
        res['article_title'],
        res['section_title'] if res['section_title'].strip() != '' else res['article_title']
    ))

Why does water heated to room temperature feel colder than the air around it?
-----

Salt fingering: 
  Salt fingering

Solar water heating: 
  Flat plate & Evacuated tube

Humidifier: 
  Fixed-installation humidifiers & Problems

Drake Landing Solar Community: 
  How it works & Energy centre

Diamond dust: 
  Characteristics & Formation

Effects of global warming on oceans: 
  Ocean currents

Mesoscale convective system: 
  Lake-effect snow

Thermal comfort: 
  Interplay of temperature and humidity

Honyaki: 
  Traditional process

Greywell Tunnel: 
  SSSI



### Training a Dense Retriever with ELI5 and in-batch Negatives
<a id='dense_train'></a>

Can we take advantage of our data to do better?

In [7]:
qar_tokenizer, qar_model = make_qa_retriever_model(
    model_name="google/bert_uncased_L-8_H-768_A-12",
    from_file=None,
    device="cuda:0"
)

### Using a Trained Dense Retriever
<a id='dense_use'></a>

Can we take advantage of our data to do better?

In [3]:
qar_tokenizer, qar_model = make_qa_retriever_model(
    model_name="google/bert_uncased_L-8_H-768_A-12",
    from_file="retriever_models/eli5_retriever_model_l-8_h-768_b-512-512_9.pth",
    device="cuda:0"
)

In [4]:
faiss_res = faiss.StandardGpuResources()
wiki40b_passage_reps = np.memmap(
            'wiki40b_passages_reps_32_l-8_h-768_b-512-512.dat',
            dtype='float32', mode='r',
            shape=(wiki40b_snippets.num_rows, 128)
)

wiki40b_index_flat = faiss.IndexFlatIP(128)
wiki40b_gpu_index = faiss.index_cpu_to_gpu(faiss_res, 1, wiki40b_index_flat)
wiki40b_gpu_index.add(wiki40b_passage_reps)

In [10]:
question = eli5['test_eli5'][12345]['title']
doc, res_list = query_qa_dense_index(
    question,
    qar_model, qar_tokenizer,
    wiki40b_snippets, wiki40b_gpu_index,
    n_results=10
)

print(question)
print('-----\n')
for res in res_list:
    print("{}: \n  {}\n".format(
        res['article_title'],
        res['section_title'] if res['section_title'].strip() != '' else res['article_title']
    ))

Why does water heated to room temperature feel colder than the air around it?
-----

Fugacity: 
  History

Heat transfer: 
  Heat transfer in the human body & Evaporative cooling

Johan Sandström: 
  Sandström  Theorem

Thermal equilibrium: 
  Bodies prepared with separately uniform temperatures, then put into purely thermal communication with each other

Evaporative cooler: 
  Physical principles

Thermal contact conductance: 
  Factors influencing contact conductance & Contact pressure

Thermodynamic temperature: 
  The heat of phase changes

Temperature: 
  Local thermodynamic equilibrium & Bodies in thermodynamic equilibrium

Tail flick test: 
  Limitations

Latent heat: 
  Usage



### Retriever Evaluation
<a id='dense_eval'></a>

How can we evaluate the embedding model? Let's start by grabbing a couple of useful metrics from the `nlp` library:

In [124]:
%%capture --no-stdout
# load the ROUGE and BERTscore metrics from the nlp library
nlp_rouge = nlp.load_metric('rouge')
nlp_bertscore = nlp.load_metric('bertscore')

# takes a list of retrieved documents and a list of possible answers
# for a question and returns a measure of the lexical overlap between the
# passages and answer
def get_aggregate_rouge(res_list, answers):
    res = np.zeros((len(res_list), len(answers), 3))
    for i, hit in enumerate(res_list):
        for j, a in enumerate(answers):
            if len(hit.strip()) > 0 and len(a.strip()) > 0:
                # get Rouge-1 P/R/F for each passage/answer pair
                score = nlp_rouge.compute([hit], [a], rouge_types=['rouge1'])['rouge1'].mid
                res[i,j] = np.array([score.precision, score.recall, score.fmeasure])
    # average P/R/F rouge scores, then find best passage-answer match
    return res.mean(axis=2).max()

# Same with BERTscore metri which aligns contextual word embedings
def get_aggregate_bertscore(res_list, answers):
    res = np.zeros((len(res_list), len(answers), 3))
    for i, hit in enumerate(res_list):
        for j, a in enumerate(answers):
            if len(hit.strip()) > 0 and len(a.strip()) > 0:
                # get Rouge-1 P/R/F for each passage/answer pair
                score = nlp_bertscore.compute([hit], [a], lang='en')
                res[i,j] = np.array([score['precision'].item(), score['recall'].item(), score['f1'].item()])
    # average P/R/F rouge scores, then find best passage-answer match
    return res.mean(axis=2).max()

# Compare which retriever finds passages that have the most
# lexical overlap with the ELI5 answers
st_time = time()
tot_rg_sparse = 0.
tot_bs_sparse = 0.
tot_rg_dense = 0.
tot_bs_dense = 0.
# ROUGE and the sparse retriver take time to compute, so we only compare
# on a small slice: we'll see that the difference is already apparent
valid_slice = eli5['validation_eli5'][:1000]
for i, (question, answers) in enumerate(zip(valid_slice['title'], valid_slice['answers'])):
    # get documents with sparse retriever
    _, sparse_res_list = query_es_index(
        question,
        es_client, index_name='wiki40b_snippets_100w',
        n_results=5
    )
    sparse_passages = [res['passage_text'] for res in sparse_res_list]
    if len(sparse_passages) == 0:
        sparse_passages = [question]
    tot_rg_sparse += get_aggregate_rouge(sparse_passages, answers['text'])
    tot_bs_sparse += get_aggregate_bertscore(sparse_passages, answers['text'])
    # get documents with dense retriever
    _, dense_res_list = query_qa_dense_index(
        question,
        qar_model, qar_tokenizer,
        wiki40b_snippets, wiki40b_gpu_index,
        n_results=5
    )
    dense_passages = [res['passage_text'] for res in dense_res_list]
    tot_rg_dense += get_aggregate_rouge(dense_passages, answers['text'])
    tot_bs_dense += get_aggregate_bertscore(dense_passages, answers['text'])
    # show average scores side by side
    if (i+1) % 10 == 0:
        print("{:03d} Sparse: RG-{:.4f} BS-{:.4f} | Dense: RG-{:.4f} BS-{:.4f} \t {:.2f}".format(
            i+1,
            tot_rg_sparse / (i+1), tot_bs_sparse / (i+1),
            tot_rg_dense / (i+1), tot_bs_dense / (i+1),
            time() - st_time
        ))

010 Sparse: RG-0.2652 BS-0.8053 | Dense: RG-0.2521 BS-0.8141 	 103.34
020 Sparse: RG-0.2657 BS-0.8068 | Dense: RG-0.2647 BS-0.8193 	 174.52
030 Sparse: RG-0.2631 BS-0.8052 | Dense: RG-0.2591 BS-0.8156 	 261.19
040 Sparse: RG-0.2623 BS-0.8049 | Dense: RG-0.2594 BS-0.8149 	 352.42
050 Sparse: RG-0.2660 BS-0.8060 | Dense: RG-0.2639 BS-0.8176 	 457.43
060 Sparse: RG-0.2698 BS-0.8053 | Dense: RG-0.2649 BS-0.8172 	 540.86
070 Sparse: RG-0.2684 BS-0.8058 | Dense: RG-0.2630 BS-0.8187 	 602.46
080 Sparse: RG-0.2671 BS-0.8062 | Dense: RG-0.2640 BS-0.8185 	 694.04
090 Sparse: RG-0.2646 BS-0.8063 | Dense: RG-0.2622 BS-0.8182 	 763.54
100 Sparse: RG-0.2627 BS-0.8058 | Dense: RG-0.2619 BS-0.8190 	 822.09
110 Sparse: RG-0.2646 BS-0.8056 | Dense: RG-0.2626 BS-0.8186 	 900.97
120 Sparse: RG-0.2673 BS-0.8055 | Dense: RG-0.2661 BS-0.8177 	 1013.28
130 Sparse: RG-0.2685 BS-0.8053 | Dense: RG-0.2678 BS-0.8175 	 1080.84
140 Sparse: RG-0.2660 BS-0.8050 | Dense: RG-0.2654 BS-0.8180 	 1380.19
150 Sparse: RG-0.

KeyboardInterrupt: 

In [123]:
sparse_res_list

[]

In [120]:
print("{:03d} Sparse: RG-{:.4f} BS-{:.4f} | Dense: RG-{:.4f} BS-{:.4f} \t {:.2f}".format(
            i+1,
            tot_rg_sparse / (i+1), tot_bs_sparse / (i+1),
            tot_rg_dense / (i+1), tot_bs_dense / (i+1),
            time() - st_time
        ))

199 Sparse: RG-0.2609 BS-0.8014 | Dense: RG-0.2634 BS-0.8135 	 1923.04


## Answer Generation Model
<a id='generation'></a>

Once we have a question and a document containing



In [2]:
from transformers import BartConfig, BartForConditionalGeneration, BartTokenizer, AdamW
from torch import nn
from typing import List
layers_to_copy = {  # maps # layers in student -> which teacher layers to copy
    6: [0, 2, 4, 7, 9, 11],
    1: [11],
    3: [0, 6, 11],
    2: [0, 11],
    4: [0, 4, 8, 11],
    9: [0, 1, 2, 4, 5, 7, 9, 10, 11],
    12: list(range(12)),
}
def init_student(student, teacher):
    """Copy everything"""
    teacher_state_dict = teacher.state_dict()
    info = student.load_state_dict(teacher_state_dict, strict=False)
    assert info.missing_keys == [], info.missing_keys
    return student, info
  
def copy_layers(teacher_layers, student_layers, l2copy: List):
    layers_to_copy = nn.ModuleList([l for i, l in enumerate(teacher_layers) if i in l2copy])
    assert len(student_layers) == len(l2copy), f"{len(student_layers)} != {len(l2copy)}"
    student_layers.load_state_dict(layers_to_copy.state_dict())
    
def make_student(teacher, student_updates):   
    d_layers_to_copy = layers_to_copy[student_updates["decoder_layers"]]
    e_layers_to_copy = layers_to_copy[student_updates["encoder_layers"]]
    kw = teacher.config.to_diff_dict()
    kw.update(student_updates)
    # Copy weights
    student_cfg = BartConfig(**kw)
    student = BartForConditionalGeneration(student_cfg)
    student, _ = init_student(student, teacher)
    copy_layers(teacher.model.encoder.layers, student.model.encoder.layers, e_layers_to_copy)
    copy_layers(teacher.model.decoder.layers, student.model.decoder.layers, d_layers_to_copy)
    return student
teacher = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
student_updates = {
    "decoder_layers": 6,
    "encoder_layers": 6,
  }
student = make_student(teacher, student_updates)

In [3]:
qa_s2s_tokenizer = BartTokenizer.from_pretrained("bart-large")
qa_s2s_model = nn.DataParallel(student.to('cuda:0'))

eli5_train_docs = json.load(open('precomputed/eli5_train_precomputed_dense_docs.json'))
eli5_valid_docs = json.load(open('precomputed/eli5_valid_precomputed_dense_docs.json'))

s2s_train_dset = ELI5DatasetS2S(eli5['train_eli5'], document_cache=dict([(k, d) for k, d, src_ls in eli5_train_docs]))
s2s_valid_dset = ELI5DatasetS2S(eli5['validation_eli5'], document_cache=dict([(k, d) for k, d, src_ls in eli5_valid_docs]), training=False)

class ArgumentsS2S():
    def __init__(self):
        self.batch_size = 4
        self.backward_freq = 16
        self.max_length = 1024
        self.print_freq = 100
        self.model_save_name = "seq2seq_models/joint_bart_student"
        self.learning_rate = 2e-4
        self.num_epochs = 20

s2s_args = ArgumentsS2S()
s2s_optimizer = AdamW(qa_s2s_model.parameters(), lr=s2s_args.learning_rate, eps=1e-8)
s2s_scheduler = get_linear_schedule_with_warmup(
        s2s_optimizer,
        num_warmup_steps=400,
        num_training_steps=s2s_args.num_epochs * math.ceil(len(s2s_train_dset) / s2s_args.batch_size)
)



In [4]:
def train_qa_s2s_epoch(model, dataset, tokenizer, optimizer, scheduler, args, e=0):
    model.train()
    # make iterator
    train_sampler = RandomSampler(dataset)
    model_collate_fn = functools.partial(
        make_qa_s2s_batch,
        tokenizer=tokenizer, max_len=args.max_length, device='cuda:0'
    )
    data_loader = DataLoader(
        dataset, batch_size=args.batch_size,
        sampler=train_sampler, collate_fn=model_collate_fn
    )
    epoch_iterator = tqdm(data_loader, desc="Iteration", disable=True)
    # accumulate loss since last print
    loc_steps = 0
    loc_loss = 0.0
    st_time = time()
    for step, batch_inputs in enumerate(epoch_iterator):
        pre_loss = model(**batch_inputs)[0]
        loss = pre_loss.sum() / pre_loss.shape[0]
        loss.backward()
        # optimizer
        if step % args.backward_freq == 0:
            optimizer.step()
            scheduler.step()
            model.zero_grad()
        # some printing within the epoch
        loc_loss += loss.item()
        loc_steps += 1
        if step % args.print_freq == 0:
            print(
                "{:2d} {:5d} of {:5d} \t L: {:.3f} \t -- {:.3f}".format(
                    e, step,
                    len(dataset) // args.batch_size,
                    loc_loss / loc_steps,
                    time() - st_time,
                )
            )
            loc_loss = 0
            loc_steps = 0

In [ ]:
s2s_args.batch_size = 4
s2s_args.print_freq = 1000

for e in range(s2s_args.num_epochs):
    train_qa_s2s_epoch(
        qa_s2s_model,
        s2s_train_dset, qa_s2s_tokenizer,
        s2s_optimizer, s2s_scheduler,
        s2s_args, e
    )
    m_save_dict = {
        'model': qa_s2s_model.state_dict(),
        'optimizer': s2s_optimizer.state_dict(),
        'scheduler': s2s_scheduler.state_dict(),
    }
    print("Saving model {}".format(s2s_args.model_save_name))
    torch.save(m_save_dict, '{}_{}.pth'.format(s2s_args.model_save_name, e))

In [29]:
torch.cuda.empty_cache()

In [10]:
def eval_qa_s2s_epoch(model, dataset, tokenizer, args):
    model.train()
    # make iterator
    train_sampler = SequentialSampler(dataset)
    model_collate_fn = functools.partial(
        make_qa_s2s_batch,
        tokenizer=tokenizer, max_len=args.max_length, device='cuda:0'
    )
    data_loader = DataLoader(
        dataset, batch_size=args.batch_size,
        sampler=train_sampler, collate_fn=model_collate_fn
    )
    epoch_iterator = tqdm(data_loader, desc="Iteration", disable=True)
    # accumulate loss since last print
    loc_steps = 0
    loc_loss = 0.0
    st_time = time()
    with torch.no_grad():
        for step, batch_inputs in enumerate(epoch_iterator):
            pre_loss = model(**batch_inputs)[0]
            loss = pre_loss.sum() / pre_loss.shape[0]
            loc_loss += loss.item()
            loc_steps += 1
            if step % args.print_freq == 0:
                print(
                    "{:5d} of {:5d} \t L: {:.3f} \t -- {:.3f}".format(
                        step,
                        len(dataset) // args.batch_size,
                        loc_loss / loc_steps,
                        time() - st_time,
                    )
                )
    print(
        "Total \t L: {:.3f} \t -- {:.3f}".format(
            loc_loss / loc_steps,
            time() - st_time,
        )
    )

In [11]:
_ = qa_s2s_model.eval()
s2s_args.print_freq = 100
eval_qa_s2s_epoch(
        qa_s2s_model,
        s2s_valid_dset, qa_s2s_tokenizer,
        s2s_args
)

    0 of  2453 	 L: 3.521 	 -- 0.315
 1000 of  2453 	 L: 3.260 	 -- 319.746
 2000 of  2453 	 L: 3.264 	 -- 638.111
Total 	 L: 3.265 	 -- 782.534


In [12]:
s2s_valid_dset[123]

('question: why is google fibre taking so long to roll out? context: <p> 2009. in march 2009, arbor worked with 100 isps to create a new network monitoring system, called atlas 2.0. in october 2009, the company estimated that google paid almost nothing for youtube\'s bandwidth, noting that google probably used dark fibre instead to run the website.\n on august 31, 2010, tektronix communications announced that it has completed its acquisition of arbor networks. upon completion of the acquisition, arbor networks joins danaher corporation\'s portfolio of communications and enterprise companies, which includes tektronix communications.\n on september 3, 2013, arbor networks announced that it had acquired privately held packetloop, a leader in security analytics <p> in january 2017, construction was halted pending concerns about the placement of google fiber huts in city parks. mayor ivy taylor expressed commitment to working with google to address community concerns and allow the project t

In [21]:
print(qa_s2s_generate(
        s2s_valid_dset[124][0], qa_s2s_model.module, qa_s2s_tokenizer,
        num_answers=1,
        num_beams=8,
        min_len=64,
        max_len=256,
        max_input_length=1024,
        device="cuda:0"
    )[0])

Yeast, yeast, and water are all part of the process of fermentation. Yeast is one of the few organisms that can make bread, beer, and wine.

When you mix yeast and water, the yeast will ferment and turn into alcohol. This is why beer is fermented and wine is fermented.


In [23]:
s2s_valid_dset[11][0]

'question: how do apps like soundhound and shazam know what song is playing? context: <p> kind of applications is mainly used for finding a song that the user does not already know. searching by sound is not limited to just identifying songs, but also for identifying melodies, tunes or advertisements, sound library management and video files.\n acoustic fingerprinting.\n the way these apps search by sound is through generating an acoustic fingerprint; a digital summary of the sound. a microphone is used to pick up an audio sample, which is then broken down into a simple numeric signature, a code unique to each track. using the same method of fingerprinting sounds, when shazam picks up a <p> such as midomi and soundhound allow users to add to that library of music in order to expand the chances to match a sound sample with its corresponding sound.\n query by humming.\n midomi and soundhound both utilize query by humming, or qbh. this is a branch off of acoustic fingerprints, but is stil